In [1]:
import sys

sys.path.append("../")

import pennylane as qml
import pennylane.numpy as np
from main import *

# np.random.seed(0)  # For reproducibility

In [2]:
num_wires = 1 
dev_ideal = qml.device("lightning.qubit", wires=num_wires )
dev_shots = qml.device("lightning.qubit", wires=num_wires, shots=100 )

In [3]:
from pennylane.measurements.probs import ProbabilityMP

In [4]:
angle = np.pi*np.random.randn()
def fun():
    qml.RY( angle , 0 )
    return ProbabilityMP( wires=[0], eigvals=np.random.rand(2) )

angle2 = np.pi*np.random.randn()
def fun2():
    qml.RY( angle2 , 0 )
    return ProbabilityMP( wires=[0], eigvals=np.random.rand(2) )

In [5]:
qml.QNode( fun, dev_ideal )()

array([0.00557084, 0.99442916])

In [6]:
qml.QNode( fun, dev_shots )()


array([0.02, 0.98])

In [7]:
def to_tape( fun ):
    with qml.tape.OperationRecorder() as tape:
        fun()   

    ops = tape.operations
    meas= tape.measurements

    tape = qml.tape.QuantumTape(
                ops=ops,
                measurements=meas ,
            )
    return tape

In [8]:
tape1 = to_tape( fun )
tape2 = to_tape( fun2 )

In [9]:
qml.execute( [tape1, tape2], device=dev_ideal )

[array([0.00557084, 0.99442916]), array([0.91243046, 0.08756954])]

In [12]:
tape1.measurements[0].eigvals()

array([0.23476531, 0.50801638])